### Train CNN to classify Find it Fix It request category based on request image

In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import os
import requests 
from multiprocessing.pool import ThreadPool

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster', rc={'font.size':35,
                              'axes.titlesize':50,
                              'axes.labelsize':35})
#sns.set_context("poster")
#sns.set_style("dark")
#sns.set_style("ticks")
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.width', 10000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.CSRRD7HKRKC3T3YXA7VY7TAZGLSWDKW6.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
#path for FIFI cleaned and preprocessed data frame
path_FIFI = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/generated_datasets/data_final.pkl'

In [3]:
#read in pkl file containing 
data = pd.read_pickle(path_FIFI)
data = pd.DataFrame(data = data)
data.reset_index(inplace=True, drop=True)

print('Original data shape:\n', data.shape, '\n')
print('Original data columns:\n', data.columns.values, '\n')

data = data[data['FIFI_category'] != 'Community Walk']

data['FIFI_category'] = data['FIFI_category'].replace({'Needles/Dumping': 'Needles_Dumping', 'Sign/Signal': 'Sign_Signal'})

data.head(1)

Original data shape:
 (248815, 67) 

Original data columns:
 ['Service_Request_Number' 'Created_Date' 'Location' 'Location_Details'
 'Description' 'License_Number' 'State' 'Vehicle_Make' 'Vehicle_Color'
 'Time_parked' 'Photo' 'FIFI_category' 'Location_Details.1'
 'Property_damage?' 'Current_Issue' 'Type_of_animal'
 'On_private_property?' 'Graffiti_location' 'Location_and_Description'
 'Obscene/Racial/Hateful?' 'Street/Intersection' 'Where?' 'Type_of_item?'
 'Veg_location' 'Vegetation_Safety_Issue' 'Parking_Violation_Concerning'
 'License_Plate_Number' 'Location_details' '7_digit_number_on_pole'
 'Issue' 'date' 'year' 'month' 'day' 'day_of_week' 'hour' 'zip'
 'neighborhood' 'SittingInformal' 'Lying' 'DisruptiveActivity_Aggressive'
 'DisruptiveActivity_Intoxicated' 'LivingActivities' 'Soliciting'
 'overdose' 'pop' 'pop_dens' 'housing_units' 'med_home' 'land_area'
 'occupied_units' 'occupied_ratio' 'med_income' 'med_age' 'male_%'
 'white_%' 'fam_%' 'nokid_%' 'mortgage_%' 'fulltime_%' 'une

,Service_Request_Number,Created_Date,Location,Location_Details,Description,License_Number,State,Vehicle_Make,Vehicle_Color,Time_parked,Photo,FIFI_category,Location_Details.1,Property_damage?,Current_Issue,Type_of_animal,On_private_property?,Graffiti_location,Location_and_Description,Obscene/Racial/Hateful?,Street/Intersection,Where?,Type_of_item?,Veg_location,Vegetation_Safety_Issue,Parking_Violation_Concerning,License_Plate_Number,Location_details,7_digit_number_on_pole,Issue,date,year,month,day,day_of_week,hour,zip,neighborhood,SittingInformal,Lying,DisruptiveActivity_Aggressive,DisruptiveActivity_Intoxicated,LivingActivities,Soliciting,overdose,pop,pop_dens,housing_units,med_home,land_area,occupied_units,occupied_ratio,med_income,med_age,male_%,white_%,fam_%,nokid_%,mortgage_%,fulltime_%,unemploy,drive_%,publictrans_%,other_%,wrk_travel,city council,county council
0,13-00036779,2013-05-09 20:32:33,"5208 BEACON AVE S, SEATTLE, WA","5200 Beacon Ave S Seattle, WA 98108; XY: 1277537.1732614825, 206078.498081425; LatLng: 47.555513, -122.302999",Ford light blue,AHZ9829,NaN,Ford,Blue,Over 3 Days,NaN,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-05-09,2013,5,9,Thursday,20,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#we are only interested in FIFI requests that have a photo included with the request
print(data.shape)
data_photo = data[~data['Photo'].isna()]
print(data_photo.shape)
data_photo.head()

(248661, 67)
(183278, 67)


,Service_Request_Number,Created_Date,Location,Location_Details,Description,License_Number,State,Vehicle_Make,Vehicle_Color,Time_parked,Photo,FIFI_category,Location_Details.1,Property_damage?,Current_Issue,Type_of_animal,On_private_property?,Graffiti_location,Location_and_Description,Obscene/Racial/Hateful?,Street/Intersection,Where?,Type_of_item?,Veg_location,Vegetation_Safety_Issue,Parking_Violation_Concerning,License_Plate_Number,Location_details,7_digit_number_on_pole,Issue,date,year,month,day,day_of_week,hour,zip,neighborhood,SittingInformal,Lying,DisruptiveActivity_Aggressive,DisruptiveActivity_Intoxicated,LivingActivities,Soliciting,overdose,pop,pop_dens,housing_units,med_home,land_area,occupied_units,occupied_ratio,med_income,med_age,male_%,white_%,fam_%,nokid_%,mortgage_%,fulltime_%,unemploy,drive_%,publictrans_%,other_%,wrk_travel,city council,county council
1,13-00064098,2013-08-13 18:05:17,"2629 SW ANDOVER ST, SEATTLE, WA","26012799 SW Andover St, Seattle; XY: 1261975.9119158287, 211063.7734459653; LatLng: 47.56828403207891, -122.3664187277485",NaN,Acxo217,NaN,Suzuki,Other,Over 1 Week,http://servicerequest.seattle.gov/media/seattle/report/photos/520ad788016358ed6cc31006/report.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-13,2013,8,13,Tuesday,18,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13-00064109,2013-08-13 18:57:44,"402 BELMONT AVE E, SEATTLE, WA","327 Belmont Ave E, Seattle; XY: 1272796.1323291115, 230309.9327528307; LatLng: 47.62162059549952, -122.3241136317349",I don't think this car has been moved in quite some time.,379 vuq,NaN,Volkswagen,Silver,Over 3 Days,http://servicerequest.seattle.gov/media/seattle/report/photos/520ae3c5016358ed6cc31054/report.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-13,2013,8,13,Tuesday,18,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13-00064128,2013-08-13 20:24:22,"5283 39TH AVE S, SEATTLE, WA","5239 39th Ave S Seattle, WA 98118; XY: 1282330.8128522313, 205697.9753960121; LatLng: 47.554657, -122.284155",Older Honda with flat tire,See photo,NaN,Honda,Blue,Over 1 Month,http://servicerequest.seattle.gov/media/seattle/report/photos/520af83b016358ed6cc31193/photo_20130813_202251.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-13,2013,8,13,Tuesday,20,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13-00064164,2013-08-14 06:51:34,"4755 38TH AVE SW, SEATTLE, WA","XY: 1258575.3814046786, 207971.16513023284; LatLng: 47.55963298157554, -122.3799168507897",Green 4 door Volkswagen Golf,WA AGD9667,NaN,Volkswagen,Green,Over 1 Month,http://servicerequest.seattle.gov/media/seattle/report/photos/520b8ae3016358ed6cc315cb/report.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-14,2013,8,14,Wednesday,6,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,13-00064186,2013-08-14 08:26:21,"4608 SW AUSTIN ST, SEATTLE, WA","7550 SW Austin St, Seattle; XY: 1255320.388053223, 199021.00473257835; LatLng: 47.53496555693373, -122.3923797794123",Sitting there so long and it hasn't moved for long time and yesterday parking enforcement didn't give ticket,76090,NaN,Dodge,Green,Over 3 Months,http://servicerequest.seattle.gov/media/seattle/report/photos/520ba12f016358ed6cc316dc/report.jpg,Abnd_Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013-08-14,2013,8,14,Wednesday,8,WA,WA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data_photo['FIFI_category'].value_counts()

Needles_Dumping    40265
Other              34428
Grafitti           31749
Abnd_Vehicle       26656
Parking            25140
Pothole            13154
Sign_Signal        8455 
Streetlight        2943 
Clogged_Drain      427  
Vegetation         44   
Dead Animal        17   
Name: FIFI_category, dtype: int64

In [ ]:
#create function to get photo from url and save

def fetch_and_save(image_url_tupple):
    image_name, image_url = image_url_tupple
    #create file name for photo
    image_name = os.path.join(save_path_cat, image_name)  
    #get photo from url and save
    r = requests.get(image_url, allow_redirects=True)
    open(image_name, 'wb').write(r.content)

save_path = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/photos/'

#get and save photos according to FIFI_category
for cat in data_photo['FIFI_category'].unique():
    
    #get data corresponding to FIFI_category
    print(cat)
    cat_data = data_photo[data_photo['FIFI_category'] == cat]
    
    #create directory for each category and save path
    os.mkdir(os.path.join(save_path, cat))
    save_path_cat = os.path.join(save_path, cat)
    
    i = 0
    while i < cat_data.shape[0]:
        print(i)
        urls = list(zip([cat+'_'+str(x)+'.jpeg' for x in np.linspace(i, i+4, num=5)], cat_data['Photo'][i:i+5]))
        ThreadPool(5).imap_unordered(fetch_and_save, urls)
        i += 5

Abnd_Vehicle
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205

8405
8410
8415
8420
8425
8430
8435
8440
8445
8450
8455
8460
8465
8470
8475
8480
8485
8490
8495
8500
8505
8510
8515
8520
8525
8530
8535
8540
8545
8550
8555
8560
8565
8570
8575
8580
8585
8590
8595
8600
8605
8610
8615
8620
8625
8630
8635
8640
8645
8650
8655
8660
8665
8670
8675
8680
8685
8690
8695
8700
8705
8710
8715
8720
8725
8730
8735
8740
8745
8750
8755
8760
8765
8770
8775
8780
8785
8790
8795
8800
8805
8810
8815
8820
8825
8830
8835
8840
8845
8850
8855
8860
8865
8870
8875
8880
8885
8890
8895
8900
8905
8910
8915
8920
8925
8930
8935
8940
8945
8950
8955
8960
8965
8970
8975
8980
8985
8990
8995
9000
9005
9010
9015
9020
9025
9030
9035
9040
9045
9050
9055
9060
9065
9070
9075
9080
9085
9090
9095
9100
9105
9110
9115
9120
9125
9130
9135
9140
9145
9150
9155
9160
9165
9170
9175
9180
9185
9190
9195
9200
9205
9210
9215
9220
9225
9230
9235
9240
9245
9250
9255
9260
9265
9270
9275
9280
9285
9290
9295
9300
9305
9310
9315
9320
9325
9330
9335
9340
9345
9350
9355
9360
9365
9370
9375
9380
9385
9390
9395
9400


15500
15505
15510
15515
15520
15525
15530
15535
15540
15545
15550
15555
15560
15565
15570
15575
15580
15585
15590
15595
15600
15605
15610
15615
15620
15625
15630
15635
15640
15645
15650
15655
15660
15665
15670
15675
15680
15685
15690
15695
15700
15705
15710
15715
15720
15725
15730
15735
15740
15745
15750
15755
15760
15765
15770
15775
15780
15785
15790
15795
15800
15805
15810
15815
15820
15825
15830
15835
15840
15845
15850
15855
15860
15865
15870
15875
15880
15885
15890
15895
15900
15905
15910
15915
15920
15925
15930
15935
15940
15945
15950
15955
15960
15965
15970
15975
15980
15985
15990
15995
16000
16005
16010
16015
16020
16025
16030
16035
16040
16045
16050
16055
16060
16065
16070
16075
16080
16085
16090
16095
16100
16105
16110
16115
16120
16125
16130
16135
16140
16145
16150
16155
16160
16165
16170
16175
16180
16185
16190
16195
16200
16205
16210
16215
16220
16225
16230
16235
16240
16245
16250
16255
16260
16265
16270
16275
16280
16285
16290
16295
16300
16305
16310
16315
16320
16325
1633

22330
22335
22340
22345
22350
22355
22360
22365
22370
22375
22380
22385
22390
22395
22400
22405
22410
22415
22420
22425
22430
22435
22440
22445
22450
22455
22460
22465
22470
22475
22480
22485
22490
22495
22500
22505
22510
22515
22520
22525
22530
22535
22540
22545
22550
22555
22560
22565
22570
22575
22580
22585
22590
22595
22600
22605
22610
22615
22620
22625
22630
22635
22640
22645
22650
22655
22660
22665
22670
22675
22680
22685
22690
22695
22700
22705
22710
22715
22720
22725
22730
22735
22740
22745
22750
22755
22760
22765
22770
22775
22780
22785
22790
22795
22800
22805
22810
22815
22820
22825
22830
22835
22840
22845
22850
22855
22860
22865
22870
22875
22880
22885
22890
22895
22900
22905
22910
22915
22920
22925
22930
22935
22940
22945
22950
22955
22960
22965
22970
22975
22980
22985
22990
22995
23000
23005
23010
23015
23020
23025
23030
23035
23040
23045
23050
23055
23060
23065
23070
23075
23080
23085
23090
23095
23100
23105
23110
23115
23120
23125
23130
23135
23140
23145
23150
23155
2316

3210
3215
3220
3225
3230
3235
3240
3245
3250
3255
3260
3265
3270
3275
3280
3285
3290
3295
3300
3305
3310
3315
3320
3325
3330
3335
3340
3345
3350
3355
3360
3365
3370
3375
3380
3385
3390
3395
3400
3405
3410
3415
3420
3425
3430
3435
3440
3445
3450
3455
3460
3465
3470
3475
3480
3485
3490
3495
3500
3505
3510
3515
3520
3525
3530
3535
3540
3545
3550
3555
3560
3565
3570
3575
3580
3585
3590
3595
3600
3605
3610
3615
3620
3625
3630
3635
3640
3645
3650
3655
3660
3665
3670
3675
3680
3685
3690
3695
3700
3705
3710
3715
3720
3725
3730
3735
3740
3745
3750
3755
3760
3765
3770
3775
3780
3785
3790
3795
3800
3805
3810
3815
3820
3825
3830
3835
3840
3845
3850
3855
3860
3865
3870
3875
3880
3885
3890
3895
3900
3905
3910
3915
3920
3925
3930
3935
3940
3945
3950
3955
3960
3965
3970
3975
3980
3985
3990
3995
4000
4005
4010
4015
4020
4025
4030
4035
4040
4045
4050
4055
4060
4065
4070
4075
4080
4085
4090
4095
4100
4105
4110
4115
4120
4125
4130
4135
4140
4145
4150
4155
4160
4165
4170
4175
4180
4185
4190
4195
4200
4205


...

24830
24835
24840
24845
24850
24855
24860
24865
24870
24875
24880
24885
24890
24895
24900
24905
24910
24915
24920
24925
24930
24935
24940
24945
24950
24955
24960
24965
24970
24975
24980
24985
24990
24995
25000
25005
25010
25015
25020
25025
25030
25035
25040
25045
25050
25055
25060
25065
25070
25075
25080
25085
25090
25095
25100
25105
25110
25115
25120
25125
25130
25135
25140
25145
25150
25155
25160
25165
25170
25175
25180
25185
25190
25195
25200
25205
25210
25215
25220
25225
25230
25235
25240
25245
25250
25255
25260
25265
25270
25275
25280
25285
25290
25295
25300
25305
25310
25315
25320
25325
25330
25335
25340
25345
25350
25355
25360
25365
25370
25375
25380
25385
25390
25395
25400
25405
25410
25415
25420
25425
25430
25435
25440
25445
25450
25455
25460
25465
25470
25475
25480
25485
25490
25495
25500
25505
25510
25515
25520
25525
25530
25535
25540
25545
25550
25555
25560
25565
25570
25575
25580
25585
25590
25595
25600
25605
25610
25615
25620
25625
25630
25635
25640
25645
25650
25655
2566

31660
31665
31670
31675
31680
31685
31690
31695
31700
31705
31710
31715
31720
31725
31730
31735
31740
31745
Needles_Dumping
0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
109

8295
8300
8305
8310
8315
8320
8325
8330
8335
8340
8345
8350
8355
8360
8365
8370
8375
8380
8385
8390
8395
8400
8405
8410
8415
8420
8425
8430
8435
8440
8445
8450
8455
8460
8465
8470
8475
8480
8485
8490
8495
8500
8505
8510
8515
8520
8525
8530
8535
8540
8545
8550
8555
8560
8565
8570
8575
8580
8585
8590
8595
8600
8605
8610
8615
8620
8625
8630
8635
8640
8645
8650
8655
8660
8665
8670
8675
8680
8685
8690
8695
8700
8705
8710
8715
8720
8725
8730
8735
8740
8745
8750
8755
8760
8765
8770
8775
8780
8785
8790
8795
8800
8805
8810
8815
8820
8825
8830
8835
8840
8845
8850
8855
8860
8865
8870
8875
8880
8885
8890
8895
8900
8905
8910
8915
8920
8925
8930
8935
8940
8945
8950
8955
8960
8965
8970
8975
8980
8985
8990
8995
9000
9005
9010
9015
9020
9025
9030
9035
9040
9045
9050
9055
9060
9065
9070
9075
9080
9085
9090
9095
9100
9105
9110
9115
9120
9125
9130
9135
9140
9145
9150
9155
9160
9165
9170
9175
9180
9185
9190
9195
9200
9205
9210
9215
9220
9225
9230
9235
9240
9245
9250
9255
9260
9265
9270
9275
9280
9285
9290


15410
15415
15420
15425
15430
15435
15440
15445
15450
15455
15460
15465
15470
15475
15480
15485
15490
15495
15500
15505
15510
15515
15520
15525
15530
15535
15540
15545
15550
15555
15560
15565
15570
15575
15580
15585
15590
15595
15600
15605
15610
15615
15620
15625
15630
15635
15640
15645
15650
15655
15660
15665
15670
15675
15680
15685
15690
15695
15700
15705
15710
15715
15720
15725
15730
15735
15740
15745
15750
15755
15760
15765
15770
15775
15780
15785
15790
15795
15800
15805
15810
15815
15820
15825
15830
15835
15840
15845
15850
15855
15860
15865
15870
15875
15880
15885
15890
15895
15900
15905
15910
15915
15920
15925
15930
15935
15940
15945
15950
15955
15960
15965
15970
15975
15980
15985
15990
15995
16000
16005
16010
16015
16020
16025
16030
16035
16040
16045
16050
16055
16060
16065
16070
16075
16080
16085
16090
16095
16100
16105
16110
16115
16120
16125
16130
16135
16140
16145
16150
16155
16160
16165
16170
16175
16180
16185
16190
16195
16200
16205
16210
16215
16220
16225
16230
16235
1624

22240
22245
22250
22255
22260
22265
22270
22275
22280
22285
22290
22295
22300
22305
22310
22315
22320
22325
22330
22335
22340
22345
22350
22355
22360
22365
22370
22375
22380
22385
22390
22395
22400
22405
22410
22415
22420
22425
22430
22435
22440
22445
22450
22455
22460
22465
22470
22475
22480
22485
22490
22495
22500
22505
22510
22515
22520
22525
22530
22535
22540
22545
22550
22555
22560
22565
22570
22575
22580
22585
22590
22595
22600
22605
22610
22615
22620
22625
22630
22635
22640
22645
22650
22655
22660
22665
22670
22675
22680
22685
22690
22695
22700
22705
22710
22715
22720
22725
22730
22735
22740
22745
22750
22755
22760
22765
22770
22775
22780
22785
22790
22795
22800
22805
22810
22815
22820
22825
22830
22835
22840
22845
22850
22855
22860
22865
22870
22875
22880
22885
22890
22895
22900
22905
22910
22915
22920
22925
22930
22935
22940
22945
22950
22955
22960
22965
22970
22975
22980
22985
22990
22995
23000
23005
23010
23015
23020
23025
23030
23035
23040
23045
23050
23055
23060
23065
2307

In [ ]:
try:
    os.mkdir('/tmp/cats-v-dogs/')
    os.mkdir('/tmp/cats-v-dogs/training/')
    os.mkdir('/tmp/cats-v-dogs/testing/')
    # Directory with our training cat/dog pictures
    os.mkdir('/tmp/cats-v-dogs/training/cats/')
    os.mkdir('/tmp/cats-v-dogs/training/dogs/')
    # Directory with our training cat/dog pictures
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass